In [2]:
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import pandas as pd
import re
# Charger les variables d'environnement
load_dotenv()

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
TIMESTAMP_FILE = 'timestamp_last_tweet.txt'

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(steam_game_id)]['success']:
            return data[str(steam_game_id)]['data']
    return None

def get_game_tags_and_check_ai(app_id):
    try:
        url = f"https://store.steampowered.com/app/{app_id}/"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Vérifier la présence de contenu généré par IA
            ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
            if ai_disclosure:
                print(f"Contenu AI détecté pour le jeu {app_id}")
                return None, None  # Le jeu utilise du contenu généré par IA
            
            # Récupérer les tags
            tag_elements = soup.find_all('a', class_='app_tag')
            tags = [tag.text.strip() for tag in tag_elements]
            return tags[:4], 'scraped'  # Retourne seulement les 4 premiers tags
    except Exception as e:
        print(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
    
    return None, None

In [3]:
get_game_details(1370110)

{'type': 'game',
 'name': 'Sarab 2: Thalj Mountains',
 'steam_appid': 1370110,
 'required_age': 0,
 'is_free': False,
 'controller_support': 'full',
 'detailed_description': '<img src="https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/1370110/extras/Baner_1.png?t=1722361210" /><br><br>Sarab 2: Thalj Mountains is the second sequence to Sarab series. It is a traditional JRPG-styled game which focuses on story-telling, exploration and turn-based battling. Each main character has its own mysterious story. Furthermore, the game has a unique atmosphere which absolutely makes it a different experience in comparison to other JRPG games.<br><br><img src="https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/1370110/extras/Baner_2.png?t=1722361210" /><br><br>After Sarab and his friends defeated one of the Five Devils and obtained the Fire Crystal, he found himself with a new adventure and a very mysterious task about missing two hearts. During their journey, Sanad a